In [25]:
import urllib.request, urllib.error, urllib.parse
import requests
import csv
from urllib.request import build_opener, HTTPCookieProcessor, Request

#PUT JOURNAL URL HERE
url = 'https://academic.oup.com/jcr/issue-archive'


opener = build_opener(HTTPCookieProcessor())
request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = opener.open(request, timeout=30)
allContent = str(response.read())

In [26]:
def toSheetHeader(name):
    # field names  
    fields = ['Title', 'Abstract','Journal']  
    
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'w') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the fields  
        csvwriter.writerow(fields)  

In [27]:
def toSheet(rows,name):
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'a') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the data rows  
        csvwriter.writerows(rows) 

In [28]:
allarticles = []


#GETTING JOURNAL TITLE


journal_index = allContent.find('<option class="header-search-bar-filters-item"')
journal_index = allContent.find('>',journal_index) + 5
journal = ""
while(allContent[journal_index]!="<"):
    journal = journal + allContent[journal_index]
    journal_index = journal_index+1


journal = journal.replace("\\n"," ")

toSheetHeader(journal)
                    
#FINDING THE LINK TO THE ISSUE, allContent is all of the code for the original URL                    
issue_index = 0

dates = ["2015","2016","2017","2018","2019","2020"]

for d in dates:
    issue_link = "https://academic.oup.com/jcr/issue-archive/"+d
    
    #getting code for ISSUE LINK here
    opener = build_opener(HTTPCookieProcessor())
    request = Request(issue_link, headers={'User-Agent': 'Mozilla/5.0'})
    response = opener.open(request, timeout=30)
    web1Content = str(response.read())

    
    #finding issue
    curr_index = 0
    while(web1Content.find('/jcr/issue/',curr_index,len(web1Content))!=-1):
        curr_index = web1Content.find('/jcr/issue/',curr_index,len(web1Content))
        
        
        iss_link_index = web1Content.find("/jcr/issue/",curr_index,len(web1Content))
        iss_link = ""
        while(web1Content[curr_index]!='"'):
            iss_link = iss_link+web1Content[curr_index]
            curr_index = curr_index+1
           
        
        opener = build_opener(HTTPCookieProcessor())
        request = Request("https://academic.oup.com"+iss_link, headers={'User-Agent': 'Mozilla/5.0'})
        response = opener.open(request, timeout=30)
        webContent = str(response.read())
    
    
        article_index = 0  
        ind = 0
        while(webContent.find('<div class="al-article-items">',article_index,len(webContent))!=-1):
            currarticle = []
            article_index = webContent.find('<div class="al-article-items">',article_index,len(webContent))
        
            link_index = webContent.find("href=",article_index,len(webContent))
            link = ""
            
            i = link_index+6
            while(webContent[i]!='"'):
                link = link+webContent[i]
                i = i+1

            title = ""
      
            i = i+2
            
            while(webContent[i]!='<'):
                title = title+webContent[i]
                i = i+1

            title = title.replace("\\n","")
            currarticle.append(title)
            #print("\033[1m",title,"\033[0;0m")

        
      
            #OPENING LINK TO ARTICLE
            opener = build_opener(HTTPCookieProcessor())
            request = Request("https://academic.oup.com"+link, headers={'User-Agent': 'Mozilla/5.0'})
            response = opener.open(request, timeout=30)
            articleContent = str(response.read())
            print("https://academic.oup.com"+link)
        
            execute = True
            abstract_index = articleContent.find('<section class="abstract">')
           
            if (abstract_index == -1):
                abstract_index = articleContent.find('article-section__content')
            if (abstract_index == -1):
                abstract_index = articleContent.find('<div class="section1')
            if (abstract_index == -1):
                execute = False
            print(abstract_index)
            
            if(execute):    
                abstract_index = articleContent.find("<p",abstract_index,len(articleContent))
                abstract_index = articleContent.find(">",abstract_index,len(articleContent))
                i = abstract_index+1
      
                end_abstract_index = articleContent.find("</p>",abstract_index,len(articleContent))

                abstract = ""

                while(i!=end_abstract_index):
                    abstract = abstract+articleContent[i]
                    i = i + 1
    
                abstract = abstract.replace("\\n"," ")
                currarticle.append(abstract)

            currarticle.append(journal)
            allarticles.append(currarticle)
            toSheet(allarticles,journal)
            allarticles=[]
            article_index = article_index+1
            issue_index = issue_index+1
            curr_index = curr_index +1


OSError: [Errno 22] Invalid argument: 'DOCTYPE html>\\r \\r  SCRAPED.csv'